## Scraping data from reddit to obtain users emotions on D.C. walkability and traffic

Connecting using credentials

In [1]:
import praw
import pandas as pd

reddit_read_only = praw.Reddit(client_id="",
							client_secret="",
							user_agent="Walkability_DC")


subreddit = reddit_read_only.subreddit("redditdev")

print("Name:", subreddit.display_name)
print("Title:", subreddit.title)
print("Description:", subreddit.description)

Name: redditdev
Title: reddit Development
Description: A subreddit for discussion of Reddit's API and Reddit API clients.

* [Read the API Overview & Rules](https://github.com/reddit/reddit/wiki/API)
* [Check out the API documentation](http://www.reddit.com/dev/api)
* [PRAW chat](https://join.slack.com/t/praw/shared_invite/enQtOTUwMDcxOTQ0NzY5LWVkMGQ3ZDk5YmQ5MDEwYTZmMmJkMTJkNjBkNTY3OTU0Y2E2NGRlY2ZhZTAzMWZmMWRiMTMwYjdjODkxOGYyZjY)
* [Snoowrap chat](https://gitter.im/not-an-aardvark/snoowrap)
* [Unofficial Discord](https://discord.gg/hVMhxpV)
* Please do not request bots here. Consider /r/requestabot instead.


Please confine discussion to Reddit's API instead of using this as a soapbox to talk to the admins. In particular, use [/r/ideasfortheadmins](/r/ideasfortheadmins) for feature ideas and [/r/bugs](/r/bugs) for bugs. If you have general reddit questions, try [/r/help](/r/help).

To see an explanation of recent user-facing changes to reddit (and the code behind them), check out /r/ch

Test to gather hot topics

In [2]:
subreddit = reddit_read_only.subreddit("Python")

for post in subreddit.hot(limit=5):
	print(post.title)
	print()

Sunday Daily Thread: What's everyone working on this week?

Saturday Daily Thread: Resource Request and Sharing! Daily Thread

Google announces the list of 574 Python packages in its new "Assured Open Source Software" service

Announcing Polyfactory - a powerful mock data generator for dataclasses, Pydantic and more

Fluke v0.3.0 has been released!



In [6]:
"""
Generating dictionary to store comments per topic
"""
bikes_urls = [
        "https://www.reddit.com/r/washingtondc/comments/tf61c3/best_dc_neighborhoods_for_biking/",
        "https://www.reddit.com/r/washingtondc/comments/3fqimr/capital_bikeshare_to_get_new_bicycles_add_nearly/",
        "https://www.reddit.com/r/bikedc/comments/10broi8/survey_wealthy_neighborhood_creates_a_very_biased/",
        "https://www.reddit.com/r/bikedc/comments/tf8272/which_dc_neighborhoods_have_the_largest/",
        "https://www.reddit.com/r/bikedc/comments/twwsyg/in_a_dc_neighborhood_group/",
        "https://www.reddit.com/r/bikedc/comments/p0ibhz/best_neighborhood_in_the_city_to_live_in_with_a/",
        "https://www.reddit.com/r/bikedc/comments/wh74p0/ebike_viability_safety_and_insurance/",
        "https://www.reddit.com/r/bikedc/comments/wtftv2/how_is_biking_in_old_town_alexandria/",
        "https://www.reddit.com/r/bikedc/comments/viztbt/safety_on_m_st_in_georgetown/",
        "https://www.reddit.com/r/bikedc/comments/uwwedy/dc_cycling_google_map_project_anyone_willing_to/",
        "https://www.reddit.com/r/bikedc/?f=flair_name%3A%22Advocacy%22",
        "https://www.reddit.com/r/bikedc/comments/pey7bz/where_to_live_for_access_to_biking/",
        "https://www.reddit.com/r/bikedc/comments/125wypf/new_old_georgetown_bike_lanes_doing_their_job/"
        ]
cars_urls = [
        "https://www.reddit.com/r/washingtondc/comments/127nkyr/the_metro_is_the_reason_my_car_stays_in_the/",
        "https://www.reddit.com/r/washingtondc/comments/z16lym/how_bad_is_traffic_out_of_dc_on_tuesday_in_the/",
        "https://www.reddit.com/r/washingtondc/comments/w1mveu/will_tripling_the_dc_traffic_cams_do_anything/",
        "https://www.reddit.com/r/washingtondc/comments/y2wbg1/dc_area_traffic_is_the_worst/",
        "https://www.reddit.com/r/washingtondc/comments/11hsl31/dc_traffic/"

]
walk_urls = [
        "https://www.reddit.com/r/washingtondc/comments/10m21lq/what_is_your_favorite_neighborhoodarea/",
        "https://www.reddit.com/r/washingtondc/comments/xyp5gy/neighborhoods_like_noma_crystal_city_are_growing/",
        "https://www.reddit.com/r/washingtondc/comments/1187atx/if_youre_willing_to_sacrifice_spaceliving_in_an/",
        "https://www.reddit.com/r/washingtondc/comments/v1qtgm/new_research_addresses_equity_in_dcs_walkability/",
        "https://www.reddit.com/r/washingtondc/comments/129raja/im_not_sure_if_some_people_on_this_subreddit/",
        "https://www.reddit.com/r/fuckcars/comments/u4fek0/i_moved_to_dc_for_the_walkability_and_im_not/",
        "https://www.reddit.com/r/washingtondc/comments/wt8r01/what_are_the_major_organizations_working_to/",
        "https://www.reddit.com/r/washingtondc/comments/ceu7oo/dc_ranks_among_most_walkable_cities_but_at_a_cost/",
        "https://www.reddit.com/r/washingtondc/comments/tjqft5/prettiest_walkable_neighborhoods_in_dc/",
        "https://www.reddit.com/r/Pokemongo_DC/comments/ysnedd/going_to_be_in_dc_by_the_mall_tomorrow_whats_the/",
        "https://www.reddit.com/r/washingtondc/comments/11lvork/what_are_your_favorite_walkable_downtowns_near_dc/",
        "https://www.reddit.com/r/washingtondc/comments/yith75/walkable_destinations_near_new_silver_line/",
        "https://www.reddit.com/r/washingtondc/comments/am680h/most_walkable_fun_dc_suburbs/",
        "https://www.reddit.com/r/washingtondc/comments/qfxhua/what_do_you_like_about_living_in_dc/",
        "https://www.reddit.com/r/washingtondc/comments/zz8i1d/trying_to_understand_why_im_struggling_to_love_dc/",
        "https://www.reddit.com/r/washingtondc/comments/10mt0u5/does_this_subreddit_represent_the_true_sentiment/",
        "https://www.reddit.com/r/washingtondc/comments/7bmyoa/the_dc_council_says_its_for_a_walkable_city_then/",
        "https://www.reddit.com/r/washingtondc/comments/ywa7ki/where_do_people_with_families_live_in_dc/"
        "https://www.reddit.com/r/washingtondc/comments/10csyr6/looking_for_neighborhoods_to_take_walks/",
]

topics = {"bikes":[bikes_urls],
          "cars":[cars_urls],
          "walk":[walk_urls],
        }

for key in topics.keys():
    comments = []
    for url in topics[key][0]:
        try:
                submission = reddit_read_only.submission(url=url)
        except:
             print("incorrect url:",url)
             continue
        submission.comments.replace_more(limit=0)
        for comment in submission.comments.list():
            comments.append(comment.body)
    topics[key].append(comments)


incorrect url: https://www.reddit.com/r/bikedc/?f=flair_name%3A%22Advocacy%22


In [7]:
#generation of dataframe with all threads per category
df_reddit = pd.DataFrame(data=[topics["bikes"][1],topics["cars"][1],topics["walk"][1]]).T
df_reddit = df_reddit.rename(columns={0:"bikes", 1:"cars", 2:"walk"})

df_reddit.head()

,bikes,cars,walk
0,Might want to repost on r/bikedc for a differe...,I probably wouldn't be living in the area if i...,I’ve had different favorites as I’ve aged. Whe...
1,Honestly the city as a whole has become very b...,I didn't even bring my car when I moved to the...,Mt. Pleasant. You go there/live there and you ...
2,"If you work downtown, there’s a protected bike...",I don't own a car. If not for the Metro (and M...,I'm surprised no one has said Capitol Hill
3,[Here’s a bike map from DDOT](https://ddot.dc....,My bike is a bigger reason my car is mostly a ...,Mt. Pleasant.\n\n&#x200B;\n\nI always said if ...
4,Are you planning to bike to work? For utility ...,"For me, it's not so much that I'm driving less...",I love Adams Morgan. There’s something for eve...


In [8]:
df_reddit.to_csv("../data/reddit.csv",index=False)

things to improve: deep into the sentiment analysis, incorporate likes to certain comments to give them more weight

important findings:

I often times see "making DC a better place to live", but the question I ask often is "for who?". For those making enough to live in the inner-most part of the city? How do lower/working class individuals work, live, and enjoy this same city when the commute to do so.. is becoming unmanageable?